In [1]:
from openpyxl import load_workbook

# Загружаем файл и нужный лист
wb = load_workbook("Все_ТС_Поездки_31.03.2025_10-22-46.xlsx")
ws = wb["Поездки"]

# Пройдемся по всем строкам, начиная с 2 (если есть заголовок)
for row in ws.iter_rows(min_row=2):
    # Столбец D (index 3) и F (index 5) — т.е. D = 4-я ячейка, F = 6-я
    for col_idx in [3, 5]:  # D и F
        cell = row[col_idx]
        if cell.hyperlink:
            cell.value = cell.hyperlink.target  # Заменяем текст на ссылку

# Сохраняем результат в новый файл
wb.save("Поездки_с_гиперссылками.xlsx")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
import pandas as pd

df = pd.read_excel('Поездки_с_гиперссылками.xlsx', sheet_name='Поездки')

pattern = r'q=(-?\d+(?:\.\d+)?),(-?\d+(?:\.\d+)?)'

df[['latitude_нач', 'longitude_нач']] = df['Нач. положение'].str.extract(pattern)

df[['latitude_нач', 'longitude_нач']] = df[['latitude_нач', 'longitude_нач']].astype(float)

df[['latitude_конеч', 'longitude_конеч']] = df['Конеч. положение'].str.extract(pattern)

df[['latitude_конеч', 'longitude_конеч']] = df[['latitude_конеч', 'longitude_конеч']].astype(float)

df = df[['№', 'Группировка', 'Начало', 'latitude_нач', 'longitude_нач', 'Конец', 'latitude_конеч', 'longitude_конеч']]

In [3]:
df

,№,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч
0,1,313 камри Сагадиев,09:55:13,51.11098,71.39509,10:21:59,51.11630,71.39345
1,1.1,2025-03-20,09:55:13,51.11098,71.39509,22:24:56,51.11107,71.39523
2,1.1.1,313 камри Сагадиев,09:55:13,51.11098,71.39509,09:59:10,51.10287,71.40412
3,1.1.2,313 камри Сагадиев,10:03:04,51.10304,71.40445,10:15:56,51.13073,71.38745
4,1.1.3,313 камри Сагадиев,10:18:41,51.13063,71.38752,10:28:15,51.11128,71.39491
...,...,...,...,...,...,...,...,...
8708,102.12.2,Джексембаев 353,09:13:33,50.02668,82.49594,09:26:56,50.07613,82.38327
8709,102.12.3,Джексембаев 353,09:29:28,50.07613,82.38327,09:31:40,50.07744,82.38622
8710,102.12.4,Джексембаев 353,09:51:18,50.07744,82.38622,09:55:02,50.07311,82.38116
8711,102.12.5,Джексембаев 353,09:58:35,50.07311,82.38116,10:04:00,50.07308,82.38129


In [4]:
df.rename(columns={'№': 'num'}, inplace=True)

df[['lvl1', 'lvl2', 'lvl3']] = df['num'].str.split('.', n=2, expand=True)

In [5]:
mask = df['lvl2'].notna()

# Применяем transform ТОЛЬКО к строкам, попадающим в mask
df.loc[mask, 'Группировка'] = (
    df[mask]
    .groupby(['lvl1', 'lvl2'])['Группировка']
    .transform('first')
)

In [6]:
# 1. Преобразуем «Начало» и «Конец» в формат времени (по умолчанию дата будет 1970-01-01)
df['Начало'] = pd.to_datetime(df['Начало'], format='%H:%M:%S', errors='coerce')
df['Конец']  = pd.to_datetime(df['Конец'],  format='%H:%M:%S', errors='coerce')

# 2. Маска: берём только те строки, у которых lvl2 не NaN/None
mask = df['lvl2'].notna()

# 3. Из «Группировка» извлекаем дату (год-месяц-день),
#    а из «Начало» и «Конец» — время (часы:минуты:секунды).
#    В итоге мы склеим в одну дату-время.

date_part = pd.to_datetime(df.loc[mask, 'Группировка'], errors='coerce').dt.date
start_times = df.loc[mask, 'Начало'].dt.time
end_times   = df.loc[mask, 'Конец'].dt.time

# 4. Склеиваем дату + время в полноценный datetime64[ns].
#    Превращаем date_part и start_times/end_times обратно в строку, чтобы сложить их через пробел.
df.loc[mask, 'Начало'] = pd.to_datetime(
    date_part.astype(str) + ' ' + start_times.astype(str),
    errors='coerce'
)
df.loc[mask, 'Конец'] = pd.to_datetime(
    date_part.astype(str) + ' ' + end_times.astype(str),
    errors='coerce'
)

In [7]:
# Пусть у вас есть столбцы: 'lvl1', 'Группировка' и т.д.

df['Группировка'] = (
    df.groupby('lvl1')['Группировка']
      .transform('first')  # берём значение из первой строки группы
)


In [8]:
df.drop('num', axis = 1, inplace=True)
df = df.dropna().reset_index(drop=True)

In [9]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

def cluster_coordinates_dbscan(df, eps=0.001):
    """
    Для каждой группы (столбец 'Группировка') с помощью DBSCAN
    группируем все координаты: и 'latitude_нач, longitude_нач', и 'latitude_конеч, longitude_конеч'.
    eps=0.0002 ~ 22 метров (т.к. 1 градус ~ 111 км).
    """

    # Делаем копию, чтобы не портить исходный DataFrame
    df_clustered = df.copy()

    # Сортируем (не обязательно, но если есть логика по времени)
    df_clustered.sort_values(by=["Группировка", "Начало"], inplace=True)

    # Разбиваем на группы
    grouped = df_clustered.groupby("Группировка", group_keys=True)

    # Сохраним результаты в список, потом склеим
    dfs = []
    for group_name, group_data in grouped:
        # Имеем один агент (группу). Нужно кластеризовать все точки:
        # как "начальные", так и "конечные".
        # Сформируем общий список координат (N*2 строки), чтобы
        # DBSCAN «видел» все точки сразу.
        
        # Собираем все "начальные" и "конечные" координаты в один массив
        # + запоминаем индексы и метку (начальная или конечная),
        # чтобы потом вернуть значения на место.
        coords = []
        index_and_type = []  # (индекс, 'start') или (индекс, 'end')

        for idx, row in group_data.iterrows():
            coords.append([row["latitude_нач"], row["longitude_нач"]])
            index_and_type.append((idx, 'start'))

            coords.append([row["latitude_конеч"], row["longitude_конеч"]])
            index_and_type.append((idx, 'end'))

        coords = np.array(coords)
        
        # Прогоняем через DBSCAN
        # eps=0.0002 -> порог, min_samples=1, чтобы из 1 точки тоже формировался кластер
        clusterer = DBSCAN(eps=eps, min_samples=1)
        labels = clusterer.fit_predict(coords)

        # Теперь у нас есть массив labels (по одной метке на каждую точку)
        # Точки с одинаковой меткой принадлежат одному кластеру.
        # Нужно заменить координаты внутри каждого кластера на "эталон" 
        # (например, на первую точку кластера или на среднее).
        
        # Определим центры кластеров как среднее по кластерам (можно брать первую точку)
        unique_labels = np.unique(labels)
        cluster_centers = {}

        for lab in unique_labels:
            cluster_points = coords[labels == lab]
            # Средняя координата внутри кластера:
            center_lat = cluster_points[:, 0].mean()
            center_lon = cluster_points[:, 1].mean()
            cluster_centers[lab] = (center_lat, center_lon)
        
        # Записываем "очищенные" координаты обратно
        for i, lab in enumerate(labels):
            (idx, coord_type) = index_and_type[i]
            new_lat, new_lon = cluster_centers[lab]

            if coord_type == 'start':
                group_data.at[idx, "latitude_нач"]   = new_lat
                group_data.at[idx, "longitude_нач"]  = new_lon
            else:  # 'end'
                group_data.at[idx, "latitude_конеч"] = new_lat
                group_data.at[idx, "longitude_конеч"] = new_lon

        dfs.append(group_data)

    # Склеиваем все обработанные группы
    df_clustered = pd.concat(dfs).sort_index()
    return df_clustered

# Пример использования:
df = cluster_coordinates_dbscan(df, eps=0.0002)

In [10]:
df.to_csv('data.csv', index=False)

In [11]:
import pandas as pd
df = pd.read_csv('data.csv')

In [12]:
pd.set_option('display.max_rows', 500)
df[df['Группировка'] == '451 Бугаев']

,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч,lvl1,lvl2,lvl3
1399,451 Бугаев,2025-03-20 08:39:03,51.126350,71.492438,2025-03-20 09:10:08,51.074005,71.766600,25,1,1
1400,451 Бугаев,2025-03-20 09:12:40,51.074005,71.766600,2025-03-20 09:42:26,51.126350,71.492438,25,1,2
1401,451 Бугаев,2025-03-21 13:02:58,51.126350,71.492438,2025-03-21 13:19:42,51.167405,71.463645,25,2,1
1402,451 Бугаев,2025-03-21 13:23:14,51.167405,71.463645,2025-03-21 13:46:46,51.114170,71.399990,25,2,2
1403,451 Бугаев,2025-03-21 13:48:36,51.113480,71.399750,2025-03-21 13:49:59,51.113190,71.402280,25,2,3
1404,451 Бугаев,2025-03-21 13:52:34,51.113190,71.402280,2025-03-21 13:54:11,51.115283,71.402100,25,2,4
1405,451 Бугаев,2025-03-21 15:47:47,51.115283,71.402100,2025-03-21 16:09:03,51.027950,71.461230,25,2,5
1406,451 Бугаев,2025-03-21 16:34:50,51.027950,71.461230,2025-03-21 16:36:45,51.028730,71.458810,25,2,6
1407,451 Бугаев,2025-03-21 16:40:30,51.028730,71.458810,2025-03-21 17:11:57,51.127830,71.467720,25,2,7
1408,451 Бугаев,2025-03-21 17:13:47,51.127650,71.468840,2025-03-21 17:20:21,51.126350,71.492438,25,2,8
